In [24]:
import pandas as pd
import mine_utils as mu
import cluster_utils as cu

In [25]:
log_path = "/Users/wvw/git/pm/ircc/lifecycles/all-17_10_25/data/level2/logs/Biographic.csv"
df = mu.get_log(log_path)

In [ ]:
# get trace sequences

filtered = cu.extract_trace_sequences(
    df,
    min_variant_freq=2
)

print("Extracted", len(filtered), "trace sequences.")
filtered.head()

Extracted 7725 trace sequences.


,trace_str,variant
case:concept:name,,
0,Submit_-_Delayed Submit Sent_-_Partner NRT,Submit_-_Delayed Submit Sent_-_Partner NRT
1,System_Error,System_Error
2,System_Error,System_Error
3,System_Error Submit Sent_-_Partner NRT,System_Error Submit Sent_-_Partner NRT
4,Submit_-_Delayed Submit Sent_-_Partner NRT,Submit_-_Delayed Submit Sent_-_Partner NRT


In [49]:
# vectorize

encoding = 'tfidf_svd'
# encoding = 'doc2vec'

encoders, artifacts = cu.vectorize(
    filtered["trace_str"],
    method=encoding
)

for name, X in encoders.items():
    print(name, X.shape)

X = encoders[encoding]
print(f"Using {encoding}:", X.shape)

tfidf_svd (7725, 10)
Using tfidf_svd: (7725, 10)


In [60]:
# cluster

# input
# cluster_algo = "kmeans"
# cluster_algo = "som"
cluster_algo = "hdbscan"

In [61]:
# find best number of clusters

if cluster_algo in [ 'kmeans', 'som' ]:
    RANGE = range(2, 16)    
    best, results = cu.find_num_clusters(
        X,
        RANGE,
        cluster_algo=cluster_algo
    )
    cu.plot_silhouette_trend(results, f"{cluster_algo.upper()} Silhouette Trend ({encoding.upper()})")
    print("highest score:", best["k"])

In [62]:
# run clustering

# input
chosen_num = 10

labels, model = cu.cluster_traces(
    X,
    num_clusters=(chosen_num if cluster_algo != "hdbscan" else None),
    cluster_algo=cluster_algo
)

cluster_stats = cu.mine_from_clusters(
    labels,
    num_clusters=chosen_num,
    sequences_df=filtered.reset_index()
)
cluster_stats

,cluster,num_traces,pct,avg_len,median_len,unique_variants
0,-1,3,0.04,7.666667,8.0,1
1,0,94,1.22,4.000000,4.0,1
2,1,108,1.40,1.000000,1.0,1
3,2,68,0.88,4.000000,4.0,1
4,3,113,1.46,5.000000,5.0,2
5,4,138,1.79,1.000000,1.0,1
6,5,117,1.51,5.000000,5.0,1
7,6,35,0.45,4.000000,4.0,1
8,7,9,0.12,8.000000,8.0,1
9,8,94,1.22,5.000000,5.0,2
